# Functions and Modules

* **Function basics**
  - signatures, returns vs. prints, docstrings, annotations.

* **Arguments**
  - positional/keyword, defaults, `*args`/`**kwargs`, keyword-only and positional-only.

* **Purity & side effects**
  - make results explicit and testable.

* **First-class functions**
  - pass/return functions; prefer comprehensions to heavy `map`/`filter` chains.

* **Modules & packages**
  - `__name__`, `python -m`, absolute imports, `src` layout, `__init__`.
 
* **Common gotchas**
  - mutable defaults, shadowing builtins, import side effects, circular imports.

## Function Basics: Signatures, Returns, Docstrings, Annotations

In [1]:
def area(w: float, h: float) -> float:
    """Rectangle area in square units."""
    return w*h

In [2]:
area(3, 4)

12

In [3]:
def greet(name: str) -> str:
    """Return a polite greeting."""
    return f"Hello, {name.title()}!"

In [4]:
greet("ada")

'Hello, Ada!'

In [5]:
def bad_add(a, b):
    print(a + b)  # side effect only

In [6]:
out = bad_add(2, 3)

5


In [7]:
out is None

True

In [8]:
def good_add(a, b):
    return a + b

In [9]:
good_add(2, 3)

5

### Arguments: Positional, Keyword, Defaults, `*args`/`**kwargs`

In [10]:
def power(x, p=2, *, verbose=False):
    result = x ** p
    if verbose:
        print(f"{x}**{p} -> {result}")  # optional logging
    return result

In [11]:
power(3), power(3, p=3)

(9, 27)

In [12]:
power(3, verbose=True)

3**2 -> 9


9

In [19]:
# convert full names to initials
[nm[0][0]+nm[1][0] for nm in [name.split() for name in ["Dragonball Zed", "Oomi Toomi"]]]

['DZ', 'OT']

In [20]:
def ratio(a, b, /, *, as_percent=False):
    q = a / b
    return f"{100*q:.1f}%" if as_percent else q

In [21]:
ratio(1, 4)

0.25

In [22]:
ratio(1, 4, as_percent=True)

'25.0%'

In [23]:
def show(*args, **kwargs):
    return args, dict(sorted(kwargs.items()))

In [24]:
show(1, 2, a=10, b=20)

((1, 2), {'a': 10, 'b': 20})

## Pure Functions vs. Side Effects

In [25]:
taxes = []

In [26]:
def tax_bad(amount, rate):
    taxes.append(amount * rate)  # hidden global mutatiion
    return amount * rate

In [27]:
def tax(amount, rate):
    return amount * rate

In [28]:
[tax(100, 0.2) for _ in range(3)]

[20.0, 20.0, 20.0]

## First-Class Functions

In [29]:
def twice(f, x):
    return f(f(x))

In [30]:
def inc(n):
    return n + 1

In [31]:
twice(inc, 3)

5

In [32]:
nums = [1, 2, 3, 4, 5]

In [33]:
list(map(lambda x: x*x, nums))

[1, 4, 9, 16, 25]

In [34]:
[x*x for x in nums]

[1, 4, 9, 16, 25]

## Closures and a Gentle Decorator

In [35]:
def make_multiplier(k):
    def mul(x):
        return k * x
    return mul

In [36]:
double = make_multiplier(2)

In [37]:
double(21)

42

In [38]:
funcs = [(lambda i=i: i*i) for i in range(3)]  # bind at definition

In [39]:
[f() for f in funcs]

[0, 1, 4]

In [40]:
import time

In [41]:
from functools import wraps

In [42]:
def timed(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        t0 = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            dt = (time.perf_counter() - t0) * 1_000
            print(f"{fn.__name__} took {dt:.2f} ms")  # timing info
    return wrapper

In [43]:
@timed
def slow_add(a, b):
    time.sleep(0.01)
    return a + b

In [44]:
slow_add(2, 3)

slow_add took 10.58 ms


5

## Modules and Packages

In [45]:
# see files in ../src

## Common Gotchas

* **Mutable defaults**

  - never use `[]` or `{}` as a default. Use `None` and create inside.
 
* **Shadowing builtins**

  - avoid names like `list`, `dict`, `sum`; you lose the original tool.
 
* **Import side effects**

  - top-level code runs on import; put work in `main()` and guard it.
 
* **Circular imports**

  - two modules importing each other can fail: move shared code to a third module.
 
* **Star imports**

  - `from x import *` pollutes namespaces; import what you use.

## Exercises

**Return, don't print**

Write `total(prices, tax=0.0)` that returns the sum of `prices` plus tax. Add a docstring and annotations.

In [47]:
def total(prices, tax=0.0):
    """
    Calculates the sum of prices pus tax.
    Args:
      prices: a list of prices for items purchased
      tax: the rate of tax to apply
    Returns:
      the total to pay
    """
    return sum(prices) * (1 + tax)

In [48]:
total([1.23, 4.56, 7.89], 0.05)

14.364

**Keyword-only option**

Implement `scale(nums, factor, *, clip=None)` that multiplies by `factor` and if `clip` is set, caps values at that maximum.

In [52]:
def scale(nums, factor, *, clip=None):
    scaled = [num*factor for num in nums]
    if clip:
        scaled = [min(s, clip) for s in scaled]
    return scaled

In [53]:
scale(range(20), 2)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]

In [54]:
scale(range(20), 2, clip=30)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 30, 30, 30, 30]

**Mutable default fixer**

Repair this function so each call starts with a fresh list: `def log(item, bucket=[]): bucket.append(item); return bucket`.

In [55]:
def log(item, bucket=None):
    bucket = [] if not bucket else bucket
    bucket.append(item)
    return bucket

In [56]:
my_bucket = ['ready']
my_bucket = log('received update', bucket=my_bucket)
print(my_bucket)

['ready', 'received update']


In [57]:
my_bucket = None
my_bucket = log('activated')
print(my_bucket)

['activated']


**Closure counter**

Write `make_counter(start=0)` returning a function that increments and returns an internal counter each call.

In [69]:
def make_counter(start=0):
    i = start
    def counter():
        nonlocal i
        i += 1
        return i
    return counter

In [70]:
count_fn = make_counter(10)
for i in range(5):
    print(count_fn())

11
12
13
14
15


In [73]:
my_counter = make_counter()
xs = [my_counter() for i in range(7)]
print(xs)

[1, 2, 3, 4, 5, 6, 7]


**Once decorator**

Create `@once` so a function runs only the first time; subsequent calls return the cached result.

In [75]:
from functools import wraps

def once(fn):
    memo = None
    @wraps(fn)
    def wrapper(*args, **kwargs):
        nonlocal memo
        if memo:
            return memo
        else:
            memo = fn(*args, **kwargs)
            return memo
    return wrapper

In [79]:
from datetime import datetime

@once
def get_start_datetime():
    current_datetime = datetime.now()
    return current_datetime

In [80]:
import time
print( get_start_datetime() )
time.sleep(10)
print( get_start_datetime() )

2025-12-14 19:17:44.312111
2025-12-14 19:17:44.312111


**Module runner**

Sketch `tools/report.py` with a `main()` and guard, then run it via `python -m tools.report`.

**Solution**

see `../src/myproj/tools/report.py`

sample run:
```
> python -m tools.report
Hello from report
```